In [1]:
import os
import sys

project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import json
from glob import glob

import pandas as pd
import torch
from IPython.display import Audio as player
from pydub import AudioSegment
from pydub.utils import which
from utils.audio import convert_mp3_to_wav, download_kaggle_dataset

/Users/julian_castro/Documents/projects/audio_app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ffmpeg_path = "/opt/homebrew/bin"

os.environ["PATH"] += os.pathsep + ffmpeg_path

AudioSegment.converter = which("ffmpeg")
AudioSegment.ffprobe = which("ffprobe")

print("ffmpeg path:", AudioSegment.converter)
print("ffprobe path:", AudioSegment.ffprobe)

path = "./../data/ludwig-music-dataset-moods-and-subgenres/mp3/mp3/rock/0A4XR1HbX46wNB1HzpmA7m.mp3"
output_path = "./../data/wav/rock/0A4XR1HbX46wNB1HzpmA7m.wav"

# Convert mp3 to wav
convert_mp3_to_wav(path, output_path)

ffmpeg path: /opt/homebrew/bin/ffmpeg
ffprobe path: /opt/homebrew/bin/ffprobe
Converted ./../data/ludwig-music-dataset-moods-and-subgenres/mp3/mp3/rock/0A4XR1HbX46wNB1HzpmA7m.mp3 to ./../data/wav/rock/0A4XR1HbX46wNB1HzpmA7m.wav


In [11]:
from utils.audio import get_audio_data
import pandas as pd

models = ["facebook/wav2vec2-base-960h"]
tracks = [
    {"id": "0A4XR1HbX46wNB1HzpmA7m", "genre": "rock"}
]

for model in models:
    data = []
    for track in tracks:
        print(f"Extracting embeddings for {model} on {track['id']} ({track['genre']})")
        track_data = get_audio_data(track["id"], track["genre"], model)
        # add track data to dataframe
        data.append(track_data)

    display(pd.DataFrame(data))


Extracting embeddings for facebook/wav2vec2-base-960h on 0A4XR1HbX46wNB1HzpmA7m (rock)
Loaded /Users/julian_castro/Documents/projects/audio_app/notebooks/utils/../../data/ludwig-music-dataset-moods-and-subgenres/mp3/mp3/rock/0A4XR1HbX46wNB1HzpmA7m.mp3 with shape (479818,)
Extracting embeddings from /Users/julian_castro/Documents/projects/audio_app/notebooks/utils/../../data/ludwig-music-dataset-moods-and-subgenres/mp3/mp3/rock/0A4XR1HbX46wNB1HzpmA7m.mp3...
With device: mps


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded /Users/julian_castro/Documents/projects/audio_app/notebooks/utils/../../data/ludwig-music-dataset-moods-and-subgenres/mp3/mp3/rock/0A4XR1HbX46wNB1HzpmA7m.mp3 with shape (479818,)
Audio tensor shape: (479818,)


,audio,embeddings,metadata,identifier
0,"[-1.3371775e-09, -2.1358315e-09, -5.8318894e-1...","[-0.004215343, 0.025813656, -0.1028079, -0.050...","{'otherSubgenres': {'L': [{'S': ' world'}, {'S...",0A4XR1HbX46wNB1HzpmA7m


In [ ]:
from transformers import AutoProcessor, ASTModel

processor = AutoProcessor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

